# 크롤링

In [1]:
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from datetime import datetime
from bs4 import BeautifulSoup as bs
import pandas as pd
import time

In [ ]:
# Chrome 옵션 설정
options = Options()
options.binary_location = "./chrome-win64/chrome.exe"
options.add_argument("--no-sandbox") # sandbox 설정
options.add_argument("--disable-dev-shm-usage") # 공유메모리 설정
options.add_argument("--window-size=1920,1080") # 화면크기설정, 반응형 웹 대비
# options.add_argument("--disable-gpu") # GPU 비활성화

# ChromeDriver 설정
service = Service("chromedriver.exe")
driver = webdriver.Chrome(service=service, options=options)
# driver.get("https://www.opinet.co.kr/searRgSelect.do")
driver.get("https://www.opinet.co.kr/")
print(driver.title)

time.sleep(3)
# 요소 찾기
hover_element = driver.find_element(By.CSS_SELECTOR, "#header > div > ul > li:nth-child(1) > a")
# menu_option = driver.find_element(By.CSS_SELECTOR, '#header > div > ul > li:nth-child(1) > ul > li:nth-child(1) > a')


# 마우스 호버 액션
actions = ActionChains(driver)
actions.move_to_element(hover_element).perform()

time.sleep(1)
driver.find_element(By.CSS_SELECTOR, "#header > div > ul > li:nth-child(1) > ul > li:nth-child(1) > a").click()

# 약간의 시간을 기다린 후 클릭
time.sleep(5)

for i in range(2,18):
    driver.find_element(By.CSS_SELECTOR, "#SIGUNGU_NM0").click()
    time.sleep(1)
    driver.find_element(By.CSS_SELECTOR, f"#SIGUNGU_NM0 > option:nth-child({i})").click()
    time.sleep(1)
    driver.find_element(By.CSS_SELECTOR, "#searRgSelect").click()
    time.sleep(1)
    driver.find_element(By.CSS_SELECTOR, "#templ_list0 > div:nth-child(7) > div > a").click()
    time.sleep(1)

driver.quit() # 브라우저 종료


# 데이터 로드

In [2]:
# !pip install hdfs

In [2]:
from hdfs import InsecureClient

In [6]:
# 파일 업로드
user = "hadoop"
host = "http://namenode:9870"

hdfs = InsecureClient(host, user)

for i in range(16):
    local_path = f'./ref/oil/지역_위치별(주유소) ({i}).xls'
    hdfs_path = f'/user/mydata/oil{i+1}.xls'
    hdfs.upload(hdfs_path, local_path, overwrite=True)
    print(f"{i+1}업로드 중..")
print("업로드 완료")

1업로드 중..
2업로드 중..
3업로드 중..
4업로드 중..
5업로드 중..
6업로드 중..
7업로드 중..
8업로드 중..
9업로드 중..
10업로드 중..
11업로드 중..
12업로드 중..
13업로드 중..
14업로드 중..
15업로드 중..
16업로드 중..
업로드 완료


In [3]:
import tempfile
import xlrd

In [6]:
# 데이터 로드
user = "hadoop"
host = "http://namenode:9870"

hdfs = InsecureClient(host, user)

path = '/user/mydata/'
datas = []
for n in range(1,17):
    file_name = f"oil{n}.xls"
    fullpath = path + file_name
    with hdfs.read(fullpath) as reader:
        with tempfile.NamedTemporaryFile(delete=False) as tmp:
            tmp.write(reader.read())  # 스트림 데이터를 임시 파일에 저장
            tmp.seek(0)  # 임시 파일의 시작점으로 이동
            data = pd.read_excel(tmp.name)  # Pandas로 임시 파일 처리
            datas.append(data)

# datas


In [7]:
for x in range(len(datas)):
    datas[x].columns = datas[x].iloc[1]
    datas[x] = datas[x].iloc[2:].reset_index(drop=True)

ori_data = pd.concat(datas, ignore_index=True)

ori_data

1,지역,상호,주소,상표,전화번호,셀프여부,고급휘발유,휘발유,경유,실내등유
0,부산광역시,신창에너지㈜직영 송정셀프주유소,부산 강서구 낙동남로 127 (송정동),S-OIL,051-972-5104,Y,-,1549,1409,-
1,부산광역시,해안주유소,부산 강서구 낙동남로 432,SK에너지,070-8894-3569,Y,-,1549,1399,-
2,부산광역시,우리2주유소,부산 강서구 낙동북로 142,알뜰주유소,051-971-5585,Y,-,1557,1417,-
3,부산광역시,동방석유㈜직영 대저주유소,부산 강서구 낙동북로 209 (대저1동),SK에너지,051-971-2135,Y,-,1557,1417,-
4,부산광역시,서강주유소,부산 강서구 낙동북로 42 (강동동),SK에너지,051-971-8484,Y,-,1557,1417,-
...,...,...,...,...,...,...,...,...,...,...
342,부산광역시,해운대주유소,부산 해운대구 좌동순환로 375,S-OIL,051-747-2526,Y,-,1633,1513,-
343,부산광역시,HD현대오일뱅크㈜직영 장산IC주유소,부산 해운대구 양운로 107 (좌동),HD현대오일뱅크,051-704-5189,N,2032,1649,1539,-
344,부산광역시,SK에너지(주)직영 해운대센텀주유소,부산 해운대구 수영강변대로 276,SK에너지,051-784-2999,Y,-,1649,1549,-
345,부산광역시,에스씨(주) 마리나타운주유소,부산 해운대구 해운대해변로 44 (우동),GS칼텍스,051-741-7030,N,1898,1698,1598,-


In [8]:
# ori_data.sample(20)

In [9]:
a_data = ori_data[['상호', '주소', '휘발유', '경유']]
a_data

1,상호,주소,휘발유,경유
0,신창에너지㈜직영 송정셀프주유소,부산 강서구 낙동남로 127 (송정동),1549,1409
1,해안주유소,부산 강서구 낙동남로 432,1549,1399
2,우리2주유소,부산 강서구 낙동북로 142,1557,1417
3,동방석유㈜직영 대저주유소,부산 강서구 낙동북로 209 (대저1동),1557,1417
4,서강주유소,부산 강서구 낙동북로 42 (강동동),1557,1417
...,...,...,...,...
342,해운대주유소,부산 해운대구 좌동순환로 375,1633,1513
343,HD현대오일뱅크㈜직영 장산IC주유소,부산 해운대구 양운로 107 (좌동),1649,1539
344,SK에너지(주)직영 해운대센텀주유소,부산 해운대구 수영강변대로 276,1649,1549
345,에스씨(주) 마리나타운주유소,부산 해운대구 해운대해변로 44 (우동),1698,1598


In [10]:
# a_data.loc[:,"지역"] = a_data["주소"].apply(lambda x: x.split(' ')[1])
# a_data

In [11]:
a_data['휘발유'].fillna(0)

0      1549
1      1549
2      1557
3      1557
4      1557
       ... 
342    1633
343    1649
344    1649
345    1698
346    1709
Name: 휘발유, Length: 347, dtype: object

In [12]:
diesel_data = a_data[["상호", "주소", "경유"]]
gasoline_data = a_data[["상호", "주소", "휘발유"]]

In [13]:
diesel_data = diesel_data.sort_values(by="경유")
gasoline_data = gasoline_data.sort_values(by="휘발유")

diesel_data["경유"] = diesel_data["경유"].replace("-", None)
gasoline_data["휘발유"] = gasoline_data["휘발유"].replace("-", None)

diesel_data.dropna(inplace=True)
gasoline_data.dropna(inplace=True)

In [14]:
diesel_data

1,상호,주소,경유
149,동방석유(주)푸른주유소,부산 동래구 충렬대로 121 (온천동),1344
148,광신석유(주)직영 충렬대로주유소,부산 동래구 충렬대로 95,1349
150,명장주유소,부산 동래구 반송로 303 (명장동),1349
151,대일주유소,부산 동래구 충렬대로 124,1349
152,㈜OS에너지 동호주유소,부산 동래구 반송로 245,1349
...,...,...,...
147,㈜삼현석유 천일주유소,부산 동구 중앙대로 168 (초량동),1698
184,대명주유소,부산 부산진구 가야대로 523 (가야동),1718
313,남항주유소,부산 영도구 남항서로 96(남항동2가),1723
314,(주)나부코직영 우원주유소,부산 영도구 태종로 181 (봉래동4가),1738


In [15]:
gasoline_data

1,상호,주소,휘발유
148,광신석유(주)직영 충렬대로주유소,부산 동래구 충렬대로 95,1489
152,㈜OS에너지 동호주유소,부산 동래구 반송로 245,1499
151,대일주유소,부산 동래구 충렬대로 124,1499
150,명장주유소,부산 동래구 반송로 303 (명장동),1499
149,동방석유(주)푸른주유소,부산 동래구 충렬대로 121 (온천동),1499
...,...,...,...
314,(주)나부코직영 우원주유소,부산 영도구 태종로 181 (봉래동4가),1848
277,㈜한국페트로 스타2주유소,부산 서구 원양로 260,1868
278,피치스주유소 송도점,부산 서구 감천로 227,1888
50,광신석유(주)직영 명지주유소,부산 강서구 르노삼성대로 372,1899


In [16]:
diesel_data['주소'] = diesel_data['주소'].replace(r'\s*\(.*?\)','', regex=True)
diesel_data

1,상호,주소,경유
149,동방석유(주)푸른주유소,부산 동래구 충렬대로 121,1344
148,광신석유(주)직영 충렬대로주유소,부산 동래구 충렬대로 95,1349
150,명장주유소,부산 동래구 반송로 303,1349
151,대일주유소,부산 동래구 충렬대로 124,1349
152,㈜OS에너지 동호주유소,부산 동래구 반송로 245,1349
...,...,...,...
147,㈜삼현석유 천일주유소,부산 동구 중앙대로 168,1698
184,대명주유소,부산 부산진구 가야대로 523,1718
313,남항주유소,부산 영도구 남항서로 96,1723
314,(주)나부코직영 우원주유소,부산 영도구 태종로 181,1738


In [17]:
gasoline_data['주소'] = gasoline_data['주소'].replace(r'\s*\(.*?\)','', regex=True)
gasoline_data

1,상호,주소,휘발유
148,광신석유(주)직영 충렬대로주유소,부산 동래구 충렬대로 95,1489
152,㈜OS에너지 동호주유소,부산 동래구 반송로 245,1499
151,대일주유소,부산 동래구 충렬대로 124,1499
150,명장주유소,부산 동래구 반송로 303,1499
149,동방석유(주)푸른주유소,부산 동래구 충렬대로 121,1499
...,...,...,...
314,(주)나부코직영 우원주유소,부산 영도구 태종로 181,1848
277,㈜한국페트로 스타2주유소,부산 서구 원양로 260,1868
278,피치스주유소 송도점,부산 서구 감천로 227,1888
50,광신석유(주)직영 명지주유소,부산 강서구 르노삼성대로 372,1899


In [58]:
diesel_ch = diesel_data.head(5)
diesel_ch

1,상호,주소,경유
149,동방석유(주)푸른주유소,부산 동래구 충렬대로 121,1344
148,광신석유(주)직영 충렬대로주유소,부산 동래구 충렬대로 95,1349
150,명장주유소,부산 동래구 반송로 303,1349
151,대일주유소,부산 동래구 충렬대로 124,1349
152,㈜OS에너지 동호주유소,부산 동래구 반송로 245,1349


In [59]:
diesel_ex = diesel_data.tail(5)
diesel_ex

1,상호,주소,경유
147,㈜삼현석유 천일주유소,부산 동구 중앙대로 168,1698
184,대명주유소,부산 부산진구 가야대로 523,1718
313,남항주유소,부산 영도구 남항서로 96,1723
314,(주)나부코직영 우원주유소,부산 영도구 태종로 181,1738
50,광신석유(주)직영 명지주유소,부산 강서구 르노삼성대로 372,1897


In [60]:
gasoline_ch = gasoline_data.head(5)
gasoline_ch

1,상호,주소,휘발유
148,광신석유(주)직영 충렬대로주유소,부산 동래구 충렬대로 95,1489
152,㈜OS에너지 동호주유소,부산 동래구 반송로 245,1499
151,대일주유소,부산 동래구 충렬대로 124,1499
150,명장주유소,부산 동래구 반송로 303,1499
149,동방석유(주)푸른주유소,부산 동래구 충렬대로 121,1499


In [61]:
gasoline_ex = gasoline_data.tail(5)
gasoline_ex

1,상호,주소,휘발유
314,(주)나부코직영 우원주유소,부산 영도구 태종로 181,1848
277,㈜한국페트로 스타2주유소,부산 서구 원양로 260,1868
278,피치스주유소 송도점,부산 서구 감천로 227,1888
50,광신석유(주)직영 명지주유소,부산 강서구 르노삼성대로 372,1899
147,㈜삼현석유 천일주유소,부산 동구 중앙대로 168,1998


In [75]:
from geopy.geocoders import Nominatim

geolocator = Nominatim(user_agent="geoapi")

def get_lat_lon(address):
    location = geolocator.geocode(address, exactly_one=False)
    if location:
        return location[0].latitude, location[0].longitude
    return None, None  # 주소를 찾지 못한 경우

gasoline_ch[['lat', 'lon']] = gasoline_ch['주소'].apply(lambda x: pd.Series(get_lat_lon(x)))
gasoline_ex[['lat', 'lon']] = gasoline_ex['주소'].apply(lambda x: pd.Series(get_lat_lon(x)))
diesel_ch[['lat', 'lon']] = diesel_ch['주소'].apply(lambda x: pd.Series(get_lat_lon(x)))
diesel_ex[['lat', 'lon']] = diesel_ex['주소'].apply(lambda x: pd.Series(get_lat_lon(x)))

C:\Users\Admin\AppData\Local\Temp\ipykernel_23872\1126667937.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gasoline_ch[['lat', 'lon']] = gasoline_ch['주소'].apply(lambda x: pd.Series(get_lat_lon(x)))
C:\Users\Admin\AppData\Local\Temp\ipykernel_23872\1126667937.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gasoline_ex[['lat', 'lon']] = gasoline_ex['주소'].apply(lambda x: pd.Series(get_lat_lon(x)))
C:\Users\Admin\AppData\Local\Temp\ipykernel_23872\1126667937.py:13: SettingWithCopyWarning: 
A value i

In [76]:
gasoline_ch

1,상호,주소,휘발유,lat,lon,type
148,광신석유(주)직영 충렬대로주유소,부산 동래구 충렬대로 95,1489,35.200766,129.089030,ch
152,㈜OS에너지 동호주유소,부산 동래구 반송로 245,1499,35.203823,129.100238,ch
151,대일주유소,부산 동래구 충렬대로 124,1499,35.206323,129.073518,ch
150,명장주유소,부산 동래구 반송로 303,1499,35.206437,129.102695,ch
149,동방석유(주)푸른주유소,부산 동래구 충렬대로 121,1499,35.200766,129.089030,ch


In [77]:
gasoline_ex

1,상호,주소,휘발유,lat,lon,type
314,(주)나부코직영 우원주유소,부산 영도구 태종로 181,1848,35.077406,129.073138,ex
277,㈜한국페트로 스타2주유소,부산 서구 원양로 260,1868,35.074528,129.006211,ex
278,피치스주유소 송도점,부산 서구 감천로 227,1888,35.078914,129.018158,ex
50,광신석유(주)직영 명지주유소,부산 강서구 르노삼성대로 372,1899,35.109622,128.931171,ex
147,㈜삼현석유 천일주유소,부산 동구 중앙대로 168,1998,35.122093,129.043398,ex


In [78]:
diesel_ch

1,상호,주소,경유,lat,lon,type
149,동방석유(주)푸른주유소,부산 동래구 충렬대로 121,1344,35.200766,129.089030,ch
148,광신석유(주)직영 충렬대로주유소,부산 동래구 충렬대로 95,1349,35.200766,129.089030,ch
150,명장주유소,부산 동래구 반송로 303,1349,35.206437,129.102695,ch
151,대일주유소,부산 동래구 충렬대로 124,1349,35.206323,129.073518,ch
152,㈜OS에너지 동호주유소,부산 동래구 반송로 245,1349,35.203823,129.100238,ch


In [79]:
diesel_ex

1,상호,주소,경유,lat,lon,type
147,㈜삼현석유 천일주유소,부산 동구 중앙대로 168,1698,35.122093,129.043398,ex
184,대명주유소,부산 부산진구 가야대로 523,1718,35.153794,129.029166,ex
313,남항주유소,부산 영도구 남항서로 96,1723,35.084001,129.036465,ex
314,(주)나부코직영 우원주유소,부산 영도구 태종로 181,1738,35.077406,129.073138,ex
50,광신석유(주)직영 명지주유소,부산 강서구 르노삼성대로 372,1897,35.109622,128.931171,ex


In [80]:
diesel_ex['type'] = "ex"
diesel_ch['type'] = "ch"


C:\Users\Admin\AppData\Local\Temp\ipykernel_23872\1008427744.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  diesel_ex['type'] = "ex"
C:\Users\Admin\AppData\Local\Temp\ipykernel_23872\1008427744.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  diesel_ch['type'] = "ch"


In [81]:
gasoline_ch['type'] = "ch"
gasoline_ex['type'] = "ex"

C:\Users\Admin\AppData\Local\Temp\ipykernel_23872\3178231797.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gasoline_ch['type'] = "ch"
C:\Users\Admin\AppData\Local\Temp\ipykernel_23872\3178231797.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gasoline_ex['type'] = "ex"


In [82]:
gasoline_result = pd.concat([gasoline_ch, gasoline_ex], axis=0, ignore_index=True)
diesel_result = pd.concat([diesel_ch, diesel_ex], axis=0, ignore_index=True)

In [101]:
gasoline_result

1,상호,주소,휘발유,lat,lon,type
0,광신석유(주)직영 충렬대로주유소,부산 동래구 충렬대로 95,1489,35.204518,129.077757,ch
1,㈜OS에너지 동호주유소,부산 동래구 반송로 245,1499,35.203823,129.100238,ch
2,대일주유소,부산 동래구 충렬대로 124,1499,35.206323,129.073518,ch
3,명장주유소,부산 동래구 반송로 303,1499,35.206437,129.102695,ch
4,동방석유(주)푸른주유소,부산 동래구 충렬대로 121,1499,35.200766,129.089030,ch
5,(주)나부코직영 우원주유소,부산 영도구 태종로 181,1848,35.077406,129.073138,ex
6,㈜한국페트로 스타2주유소,부산 서구 원양로 260,1868,35.074528,129.006211,ex
7,피치스주유소 송도점,부산 서구 감천로 227,1888,35.078914,129.018158,ex
8,광신석유(주)직영 명지주유소,부산 강서구 르노삼성대로 372,1899,35.109622,128.931171,ex
9,㈜삼현석유 천일주유소,부산 동구 중앙대로 168,1998,35.122093,129.043398,ex


In [102]:
diesel_result

1,상호,주소,경유,lat,lon,type
0,동방석유(주)푸른주유소,부산 동래구 충렬대로 121,1344,35.204518,129.077757,ch
1,광신석유(주)직영 충렬대로주유소,부산 동래구 충렬대로 95,1349,35.200766,129.089030,ch
2,명장주유소,부산 동래구 반송로 303,1349,35.206437,129.102695,ch
3,대일주유소,부산 동래구 충렬대로 124,1349,35.206323,129.073518,ch
4,㈜OS에너지 동호주유소,부산 동래구 반송로 245,1349,35.203823,129.100238,ch
5,㈜삼현석유 천일주유소,부산 동구 중앙대로 168,1698,35.122093,129.043398,ex
6,대명주유소,부산 부산진구 가야대로 523,1718,35.153794,129.029166,ex
7,남항주유소,부산 영도구 남항서로 96,1723,35.084001,129.036465,ex
8,(주)나부코직영 우원주유소,부산 영도구 태종로 181,1738,35.077406,129.073138,ex
9,광신석유(주)직영 명지주유소,부산 강서구 르노삼성대로 372,1897,35.109622,128.931171,ex


In [99]:

loc = gasoline_result.iloc[0, 1]
geolocator = Nominatim(user_agent="geoapi")
location = geolocator.geocode(loc, exactly_one=False)

gasoline_result.iloc[0,3] = location[-1].latitude
gasoline_result.iloc[0,4] = location[-1].longitude


In [100]:
loc = diesel_result.iloc[1, 1]
geolocator = Nominatim(user_agent="geoapi")
location = geolocator.geocode(loc, exactly_one=False)

diesel_result.iloc[0,3] = location[-1].latitude
diesel_result.iloc[0,4] = location[-1].longitude

In [108]:
import folium


coordinates = gasoline_result[['lat', 'lon']].to_numpy()
place = gasoline_result['상호'].to_numpy()

# 중심좌표 설정(기본위치)
center_lat, center_lon = 35.17574571, 129.0817409

# 지도생성
m = folium.Map(location=[center_lat, center_lon], zoom_start=11.5)
store_type = gasoline_result["type"]

for i, (lat, lon) in enumerate(coordinates):
    if store_type[i] == "ch":
        ccolor = "green"
        fcolor = "yellow"
    elif store_type[i] == "ex":
        ccolor = "red"
        fcolor = "orange"
    else:
        ccolor = "blue"
        fcolor = "skyblue"
    
    folium.CircleMarker(
        location=[lat, lon],
        radius=10,
        color= ccolor,
        fill = True,
        fill_color = fcolor,
        fill_opacity = 0.5,
        tooltip = place[i]
    ).add_to(m) # 지도 생성 후 원을 그리고 지도에 붙임


    folium.Marker(
        location=[lat + 0.001, lon],
        icon=folium.DivIcon(f'<div style="font-size: 9px; color: black; background-color: white; width:50px; white-space:nowrap; overflow:hidden; text-overflow: ellipsis; opacity:0.7; border-radius:5px">{place[i]}</div>')
    ).add_to(m)

m.save("./휘발유.html")   
m

In [107]:


coordinates = diesel_result[['lat', 'lon']].to_numpy()
place = diesel_result['상호'].to_numpy()

# 중심좌표 설정(기본위치)
center_lat, center_lon = 35.17574571, 129.0817409

# 지도생성
m = folium.Map(location=[center_lat, center_lon], zoom_start=11.5)
store_type = diesel_result["type"]

for i, (lat, lon) in enumerate(coordinates):
    if store_type[i] == "ch":
        ccolor = "green"
        fcolor = "yellow"
    elif store_type[i] == "ex":
        ccolor = "red"
        fcolor = "orange"
    else:
        ccolor = "blue"
        fcolor = "skyblue"
    
    folium.CircleMarker(
        location=[lat, lon],
        radius=10,
        color= ccolor,
        fill = True,
        fill_color = fcolor,
        fill_opacity = 0.5,
        tooltip = place[i]
    ).add_to(m) # 지도 생성 후 원을 그리고 지도에 붙임


    folium.Marker(
        location=[lat + 0.001, lon],
        icon=folium.DivIcon(f'<div style="font-size: 9px; color: black; background-color: white; width:50px; white-space:nowrap; overflow:hidden; text-overflow: ellipsis; opacity:0.7; border-radius:5px">{place[i]}</div>')
    ).add_to(m)

m.save("./경유.html")   
m

In [87]:
loc = "부산 동래구 명장동 319-6"
geolocator = Nominatim(user_agent="geoapi")
location = geolocator.geocode(loc)
print(location.latitude, location.longitude)

35.20755 129.10356


In [88]:
loc = "부산 동래구 안락동 946-6"
# geolocator = Nominatim(user_agent="geoapi")
location = geolocator.geocode(loc)
print(location.latitude, location.longitude)

35.1955604 129.1062237


In [89]:
loc = "부산 동래구 반송로 245"
geolocator = Nominatim(user_agent="geoapi")
location = geolocator.geocode(loc, exactly_one=False)
print(location[0].latitude, location[0].longitude)

35.2038232 129.1002378


In [90]:
loc = "부산 동래구 반송로 303"
geolocator = Nominatim(user_agent="geoapi")
location = geolocator.geocode(loc)
print(location.latitude, location.longitude)

35.1961567 129.0940769


In [91]:
geolocator = Nominatim(user_agent="geoapi")
location = geolocator.geocode("부산 동래구 반송로 245")
print(location.raw)

{'place_id': 250765094, 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. http://osm.org/copyright', 'osm_type': 'way', 'osm_id': 166596757, 'lat': 35.2084122, 'lon': 129.1030132, 'class': 'highway', 'type': 'residential', 'place_rank': 26, 'importance': 0.0534055336842908, 'addresstype': 'road', 'name': '반송로', 'display_name': '반송로, 건물, 명장2동, 동래구, 부산광역시, 47764, 대한민국', 'boundingbox': [35.2083857, 35.2084407, 129.102475, 129.1035514]}
